## Running a simple optimization with `jax_md`

In [1]:
import os
import sys
sys.path.append(os.path.join(os.getcwd(), "../../.."))

In [2]:
import functools
from pathlib import Path

import jax
import jax.numpy as jnp
import jax_md
import optax

import jax_dna.energy.dna1 as jdna_energy
import jax_dna.input.topology as jdna_top
import jax_dna.input.trajectory as jdna_traj
import jax_dna.losses.observable_wrappers as jdna_losses
import jax_dna.observables as jdna_obs
import jax_dna.simulators.jax_md as jdna_jaxmd
import jax_dna.ui.loggers.jupyter as jupyter_logger
import jax_dna.utils.types as jdna_types

jax.config.update("jax_enable_x64", True)

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


This is will be similar to the simple simulation except we arr going to add a loss and optimize the parameters of the simulation. 

In [3]:
run_config = {
    "n_sim_steps": 20_000,
    "n_opt_steps": 25,
    "learning_rate": 0.001,
}

In [4]:
experiment_dir = Path("../../../data/sys-defs/simple-helix")

topology = jdna_top.from_oxdna_file(experiment_dir / "sys.top")
initial_positions = (
    jdna_traj.from_file(
        experiment_dir / "bound_relaxed.conf",
        topology.strand_counts,
    )
    .states[0]
    .to_rigid_body()
)

experiment_config, energy_config = jdna_energy.default_configs()

dt = experiment_config["dt"]
kT = experiment_config["kT"]
diff_coef = experiment_config["diff_coef"]
rot_diff_coef = experiment_config["rot_diff_coef"]

# These are special values for the jax_md simulator
gamma = jax_md.rigid_body.RigidBody(
    center=jnp.array([kT / diff_coef], dtype=jnp.float64),
    orientation=jnp.array([kT / rot_diff_coef], dtype=jnp.float64),
)
mass = jax_md.rigid_body.RigidBody(
    center=jnp.array([experiment_config["nucleotide_mass"]], dtype=jnp.float64),
    orientation=jnp.array([experiment_config["moment_of_inertia"]], dtype=jnp.float64),
)

geometry = energy_config["geometry"]
transform_fn = functools.partial(
    jdna_energy.Nucleotide.from_rigid_body,
    com_to_backbone=geometry["com_to_backbone"],
    com_to_hb=geometry["com_to_hb"],
    com_to_stacking=geometry["com_to_stacking"],
)

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


The jax_md simulator needs an energy function. We can use the default
energy functions and configurations for dna1 simulations. For more
information on energy functions and configurations, see the documentation.

In [5]:
energy_fn_configs = jdna_energy.default_energy_configs()

For this example were only going to optimize the parameters that are
associated with the Stacking energy function.

In [6]:
params = []
for ec in energy_fn_configs:
    params.append(
        ec.opt_params if isinstance(ec, jdna_energy.StackingConfiguration) else {}
    )
# we're not going to optimize wrt the seq specific stacking weights
for op in params:
    if "ss_stack_weights" in op:
        del op["ss_stack_weights"]

In [7]:
energy_fns = jdna_energy.default_energy_fns()

simulator = jdna_jaxmd.JaxMDSimulator(
    energy_configs=energy_fn_configs,
    energy_fns=energy_fns,
    topology=topology,
    simulator_params=jdna_jaxmd.StaticSimulatorParams(
        seq=jnp.array(topology.seq),
        mass=mass,
        bonded_neighbors=topology.bonded_neighbors,
        checkpoint_every=500,
        dt=dt,
        kT=kT,
        gamma=gamma,
    ),
    space=jax_md.space.free(),
    transform_fn=transform_fn,
    simulator_init=jax_md.simulate.nvt_langevin,
    neighbors=jdna_jaxmd.NoNeighborList(unbonded_nbrs=topology.unbonded_neighbors),
)

The ObservableLossFn class is a convenience wrapper for computing the the
loss of an observable. In this case, we are using the propeller twist and
the loss is squared error. the ObservableLossFn class implements `__call__`
that takes the output of the simulation, the target, and weights and
returns the loss and the measured observable.

In [8]:
loss_fn = jdna_losses.ObservableLossFn(
    observable=jdna_obs.propeller.PropellerTwist(
        rigid_body_transform_fn=transform_fn,
        h_bonded_base_pairs=jnp.array([[1, 14], [2, 13], [3, 12], [4, 11], [5, 10], [6, 9]])
    ),
    loss_fn=jdna_losses.RootMeanSquaredError(),
    return_observable=True,
)

# we're going to ignore the first 10% of the simulation steps for the loss calculation
eq_steps = int(run_config["n_sim_steps"] * 0.1)
other_steps = run_config["n_sim_steps"] - eq_steps
weights = jnp.concat([
    jnp.zeros(eq_steps, dtype=jnp.float64),
    jnp.ones(other_steps, dtype=jnp.float64)/other_steps
])

# target_prop_twist = jnp.array(jdna_obs.propeller.TARGETS["oxDNA"], dtype=jnp.float64)
target_prop_twist = jnp.array(20.0, dtype=jnp.float64)
def graddable_loss(in_params:jdna_types.Params, in_key:jax.random.PRNGKey) -> tuple[float, tuple[float, jax.random.PRNGKey]]:
    in_key, subkey = jax.random.split(in_key)
    sim_out = simulator.run(in_params, initial_positions, run_config["n_sim_steps"], subkey)
    loss, ptwist = loss_fn(sim_out, target_prop_twist, weights)
    return (loss, (ptwist, in_key))

grad_fn = jax.jit(jax.value_and_grad(graddable_loss, has_aux=True))

In [9]:
logger = jupyter_logger.JupyterLogger(
    simulators=[],
    observables=[],
    objectives=[],
    metrics_to_log=["loss", ["prop_twist", "target_ptwist"]],# + params_to_log,
    max_opt_steps=run_config["n_opt_steps"],
    plots_size_px=(600, 1200),
    plots_nrows_ncols = (2, 1)
)
logger.show()

/home/ryan/repos/jax-dna/examples/simple_optimizations/jaxmd/../../../jax_dna/ui/loggers/logger.py:33: UserWarning: `log_dir` not results might not be saved to disk.
  warnings.warn(MISSING_LOGDIR_WANING, stacklevel=1)


In [10]:
key = jax.random.PRNGKey(1234)
optimizer = optax.adam(learning_rate=run_config["learning_rate"])
opt_state = optimizer.init(params)

for i in range(run_config["n_opt_steps"]):
    (loss, (prop_twist, _)), grads = grad_fn(params, key)

    logger.log_metric("loss", loss, i)
    logger.log_metric("prop_twist", prop_twist, i)
    logger.log_metric("target_ptwist", target_prop_twist, i)

    updates, opt_state = optimizer.update(grads, opt_state)
    params = optax.apply_updates(params, updates)
    logger.increment_prog_bar()